In [1]:
from __future__ import print_function, division
import matplotlib.pyplot as plt
import pandas as pd

from neuralnilm.data.loadactivations import load_nilmtk_activations
from neuralnilm.data.syntheticaggregatesource import SyntheticAggregateSource
from neuralnilm.data.datapipeline import DataPipeline
from neuralnilm.data.processing import DivideBy
from neuralnilm.data.datathread import DataThread


In [2]:
NILMTK_FILENAME = '/data/mine/vadeec/merged/ukdale.h5'
TARGET_APPLIANCE = 'kettle'
SEQ_LENGTH = 2048
SAMPLE_PERIOD = 6
STRIDE = 4

LOADER_CONFIG = {
    'nilmtk_activations': dict(
        appliances=['kettle', 'microwave', 'washing machine'],
        filename=NILMTK_FILENAME,
        sample_period=SAMPLE_PERIOD,
        windows={
            'train': {
                1: ("2014-01-01", "2014-02-01")
            },
            'unseen_activations_of_seen_appliances': {
                1: ("2014-02-02", "2014-02-08")                
            },
            'unseen_appliances': {
                2: ("2013-06-01", "2013-06-07")
            }
        }
    )
}

nilmtk_activations = load_nilmtk_activations(**LOADER_CONFIG['nilmtk_activations'])

INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 112 kettle activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 114 microwave activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading washing machine for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 23 washing machine activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 31 kettle activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 31 microwave activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading washing machine for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations

In [3]:
nilmtk_activations.keys()

['train', 'unseen_activations_of_seen_appliances', 'unseen_appliances']

In [4]:
source = SyntheticAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH
)

In [24]:
#FOLD = 'train'
#FOLD = 'unseen_activations_of_seen_appliances'
FOLD = 'unseen_appliances'
seq = source.get_sequence(enable_all_appliances=True)

In [25]:
all(seq.all_appliances.sum(axis=1) == seq.input[:, 0])

True

In [26]:
fig, axes = plt.subplots(2, sharex=True)
seq.all_appliances.plot(ax=axes[0])
axes[1].plot(seq.input)
fig.suptitle(FOLD)
plt.show()

In [28]:
sample = source.get_batch(num_seq_per_batch=1024)

In [30]:
pipeline = DataPipeline(
    source, 
    num_seq_per_batch=64,
    target_processing=[DivideBy(sample.before_processing.target.flatten().std())]
)

# Disaggregation

In [ ]:
nilmtk_disag_source = NILMTKDisagSource(
    filename=NILMTK_FILENAME,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    buildings=[5],
    window_per_building={},
    stride=STRIDE,
    sample_period=SAMPLE_PERIOD
)

In [ ]:
disag_pipeline = deepcopy(pipeline)
disag_pipeline.source = nilmtk_disag_source

In [ ]:
disaggregator = Disaggregator(
    pipeline=disag_pipeline,
    output_path=PATH  # 
)

Disagregator ideas:

* make a copy of pipeline but swap source for a NILMTKDisagSource
* NILMTKDisagSource loads all data into memory (?) and iterates over chunks of it (get seq_length from pipeline.source.seq_length)
* 

In [31]:
report = pipeline.report()
report['activations'] = LOADER_CONFIG
report

{'DataPipeline': {'input_processing': [],
  'num_seq_per_batch': 64,
  'source': {'SyntheticAggregateSource': {'allow_incomplete_distractors': True,
    'allow_incomplete_target': True,
    'distractor_inclusion_prob': 1.0,
    'include_incomplete_target_in_output': True,
    'rng_seed': None,
    'seq_length': 2048,
    'target_appliance': 'kettle',
    'target_inclusion_prob': 1.0,
    'uniform_prob_of_selecting_each_model': True}},
  'target_processing': [{'DivideBy': {'divisor': 218.98782}}]},
 'activations': {'nilmtk_activations': {'appliances': ['kettle',
    'microwave',
    'washing machine'],
   'filename': '/data/mine/vadeec/merged/ukdale.h5',
   'sample_period': 6,
   'windows': {'train': {1: ('2014-01-01', '2014-02-01')},
    'unseen_activations_of_seen_appliances': {1: ('2014-02-02', '2014-02-08')},
    'unseen_appliances': {2: ('2013-06-01', '2013-06-07')}}}}}

In [ ]:
from lasagne.layers import InputLayer, RecurrentLayer, DenseLayer, ReshapeLayer

def get_net_0(input_shape, target_shape=None):
    NUM_UNITS = {'dense_layer_0': 100}
    if target_shape is None:
        target_shape = input_shape
    
    # Define layers
    input_layer = InputLayer(
        shape=input_shape
    )
    dense_layer_0 = DenseLayer(
        input_layer, 
        num_units=NUM_UNITS['dense_layer_0']
    )
    final_dense_layer = DenseLayer(
        dense_layer_0,
        num_units=target_shape[1] * target_shape[2],
        nonlinearity=None
    )
    output_layer = ReshapeLayer(
        final_dense_layer,
        shape=target_shape
    )
    return output_layer

In [ ]:
from neuralnilm.net import Net

batch = pipeline.get_batch()
output_layer = get_net_0(
    batch.after_processing.input.shape, 
    batch.after_processing.target.shape
)
net = Net(output_layer)

In [ ]:
from neuralnilm.trainer import Trainer

PATH = '/home/dk3810/temp/neuralnilm'

trainer = Trainer(
    net,
    data_pipeline=pipeline,
    path=PATH,
    learning_rate=1E-4,
    repeat_callbacks=[
        (  10, Trainer.validate),
        (  10, Trainer.plot_costs)
    ]
)

In [ ]:
trainer.fit(50)

In [ ]:
trainer.iteration

In [ ]:
trainer.training_costs